# install dependencies

In [ ]:
!pip install -q torch transformers transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl pacmap datasets langchain-community ragatouille langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Imports and Initial Setup

In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt

pd.set_option(
    "display.max_colwidth", None
)  # This will be helpful when visualizing retriever outputs

# Data Loading and Preprocessing

In [ ]:
# Read the CSV file
file_path = "study_results_summary-chatGPTcorrect.xlsx"  # Update with your file path
df = pd.ExcelFile(file_path).parse("Sheet3")
# df=df[df["Labeling_State"]=="Gold Standard"]
# df=df[df["Correct_Label"].str.contains("2 - the study's primary" )]

In [ ]:
df.columns=[i.replace(" ","_") for i in list(df)]

# AzureML Endpoint Setup

In [ ]:
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLChatOnlineEndpoint,
    AzureMLEndpointApiType,
    CustomOpenAIChatContentFormatter
)
from typing import List, Optional, Tuple

# Define the AzureMLChatOnlineEndpoint
azure_model = AzureMLChatOnlineEndpoint(
    endpoint_url="https://Meta-Llama-3-70B-Instruct-vycmo-serverless.eastus2.inference.ai.azure.com/v1/chat/completions",
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key="",
    content_formatter=CustomOpenAIChatContentFormatter(),
)

## usage example

In [ ]:


# Define the answer_with_rag function
def LLM(
    question: str,
    azure_model: AzureMLChatOnlineEndpoint,


    num_retrieved_docs: int = 30,
    num_docs_final: int = 5) -> Tuple[str, List[str]]:

    # Gather documents with retriever
    print("=> Retrieving documents...")

    # Build the final prompt
    context = "\nExtracted documents:\n"
    # context += " ".join(
    #     [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    # )


    final_prompt_template = [
        {
            "role": "system",
            "content": f"""You are a medical researcher extracting specific information from scientific literature.

    When provided with an abstract and a question, your task is to extract the specific information requested.

    **Guidelines:**

    - Use only the information provided in the abstract.
    - **Provide only the answer to the question, without any additional text or explanations.**
    - Do **not** include phrases like "The answer is", "According to the abstract", etc.
    - If the answer is not specified in the abstract, write "Not specified."
    - Ensure your response is concise and contains only the information asked for.
    - for duration, use these example format: 30 years (1984 - 2014), 15 years (1996 - 2011), ~18 years(January 2003 - March 2021), 1.5 years (starting August 2020)
    - for participant number, give format like this example: 120
    - clinical outcomes, give meaningful information, for example: - Median gamma glutamyl transferase decreased from 221 U/L to 104 U/L by 1 year post-diagnosis (P<0.0001),
      indicating improved liver function.\n- Transplant-free survival at 10 years was 89%.\n- Lower liver transplantation rate in PSC-IBD patients compared to those without
      IBD (2% vs 18%, P=0.01).\n- Cirrhosis rate was lower in patients diagnosed with IBD before PSC (15% vs 31%, P=0.05).
    - study purpose: remember to include the disease name
    - inclusion criteria: includes all conditions, bedetailed. for example, Patients diagnosed with HHT based on three or four Curaçao criteria (specific criteria not detailed in the abstract), if subcriteria details not specified.


    """,
        },
        {
            "role": "user",
            "content": f"""

    Question: {question}, retuen the answer as a string. No NEED TO include THE QUESTION AGAIN IN THE ANSWER
""",
        },
    ]


    # Use the AzureMLChatOnlineEndpoint model to get the response
    print("=> Generating answer...")
    response = azure_model.invoke(final_prompt_template)

    if isinstance(response, (tuple, list)):
        response_message = response[0]  # Adjust based on the actual structure
        answer = response_message.content
    else:
        answer = response.content

    return answer


In [ ]:
question = "study purpose for paper with title The Natural History of Primary Sclerosing Cholangitis in Children: A Large Single-Center Longitudinal Cohort Study?"

answer = LLM(
    question, azure_model
)

=> Retrieving documents...
=> Generating answer...


In [ ]:
print("-----------------------------------Answer-----------------------------------")
print(f"{answer}")


-----------------------------------Answer-----------------------------------
To describe the natural history of primary sclerosing cholangitis (PSC) in children.


# Run data

In [ ]:
questions_list = [
    "study purpose",
    "type of study",
    "duration of the study",
    "participants bumbers in study",
    "data collection occur",
    "inclusion criteria for the study",
    "exclusion criteria for the study",
    "clinical outcomes",
    "differed between the treatments",
    "rare disease associated with the study",
    "Location where the NHS was conducted"
]





In [ ]:
# Read the CSV file
file_path = "all_2023_04_07.xlsx"  # Update with your file path
dfg = pd.ExcelFile(file_path).parse("all_2023_04_07")
dfg=dfg[dfg["Labeling_State"]=="Gold Standard"]
dfg=dfg[dfg["Correct_Label"].str.contains("2 - the study's primary" )]

In [ ]:
# df_temp=dfg.head(15).copy()
df_temp=dfg.copy()

In [ ]:
df_temp.shape


(148, 10)

In [ ]:
from tqdm import tqdm
import urllib.request
from tenacity import retry, stop_after_attempt, wait_fixed
import time
# Configure retry behavior
max_retries = 2
retry_interval = 3  # seconds

@retry(stop=stop_after_attempt(max_retries), wait=wait_fixed(retry_interval))
def LLM_with_retry(question, azure_model):
  answer = LLM(question, azure_model )
  return answer

for question in questions_list:

  for index, row in tqdm(df_temp.iterrows(), total=len(df_temp)):  # Add tqdm here

      abstarct=row["ABSTRACT"]
      question1 =   f" This is the abstarct {abstarct}, please answer for this question {question}."
      try:
        try:
            answer = LLM_with_retry(
                question1, azure_model
            )
            df_temp.loc[index,question] = answer
            time.sleep(1)
        except urllib.error.URLError as e:
            if isinstance(e.reason, socket.timeout):
                print(f"Request timed out for question: {question1}")
                df_temp.loc[index,question] = "time-out error"
                time.sleep(1)
            else:
                print(f"Error making request: {e}")
                df_temp.loc[index,question] = "request error"
                time.sleep(1)
      except Exception as e:
          print(f"Error processing question: {question1}, Error: {e}")
          df_temp.loc[index,question] = "processing error"
          time.sleep(1)

  0%|          | 0/148 [00:00<?, ?it/s]

=> Retrieving documents...
=> Generating answer...


  1%|          | 1/148 [00:03<08:24,  3.43s/it]

=> Retrieving documents...
=> Generating answer...


  1%|▏         | 2/148 [00:08<10:04,  4.14s/it]

...

 99%|█████████▊| 146/148 [05:50<00:04,  2.33s/it]

=> Retrieving documents...
=> Generating answer...


 99%|█████████▉| 147/148 [05:53<00:02,  2.41s/it]

=> Retrieving documents...
=> Generating answer...


100%|██████████| 148/148 [05:55<00:00,  2.41s/it]


In [ ]:
df_temp.to_csv("Azure_LLAMA3_RAG_abstracts_07242024_10292024_all-no_rag_questions_a.csv")

In [ ]:
df_temp.shape


(148, 21)

In [ ]:
df_temp

,PMID,Labeling_State,Correct_Label,Agreement,Explanation,TITLE,ABSTRACT,vector,curated,y,...,type of study,duration of the study,participants bumbers in study,data collection occur,inclusion criteria for the study,exclusion criteria for the study,clinical outcomes,differed between the treatments,rare disease associated with the study,Location where the NHS was conducted
0,27504812,Gold Standard,'2 - the study's primary contribution centers on observing the time course of a rare disease',0.900,NaN,The Natural History of Primary Sclerosing Cholangitis in Children: A Large Single-Center Longitudinal Cohort Study,"Data regarding pediatric primary sclerosing cholangitis (PSC) natural history are limited. We describe a large pediatric PSC cohort with longitudinal follow-up. The present study records review of pediatric patients with PSC diagnosed between 1984 and 2014. N‚Ää=‚Ää120 (63% M) ages 1 to 21 years (median 14 years) at diagnosis. 27% (31/113) had autoimmune sclerosing cholangitis (ASC), 24% had exclusive small duct PSC, METAVIR stage was F3-F4 in 41%. Eighty-one percent of patients with PSC had inflammatory bowel disease (IBD); most had ulcerative/indeterminate colitis (72/97), typically pancolitis (40/72). PSC-IBD was more common than ASC-IBD (85% vs 68%, P‚Ää=‚Ää0.03). Median follow-up was 3.7 years (interquartile range [IQR] 1.5, 6.9). Median gamma glutamyl transferase decreased from baseline of 221 U/L (IQR 110, 425) to 104 U/L by 1 year postdiagnosis ([IQR 18,229], P‚Ää<‚Ää0.0001), and then changed little. Mean fibrosis stage at diagnosis was 2.3‚Ää¬±‚Ää1.4 (N‚Ää=‚Ää91), and at 1 to 5 years was 2.6‚Ää¬±‚Ää1.3 (N‚Ää=‚Ää20). Transplant-free survival at 10 year was 89%; there were 6 liver transplants, 2 in patients with small duct PSC and 4 with diffuse large duct PSC. Although the cirrhosis rate was not significantly different in PSC with IBD versus without (22% vs 41%, P‚Ää=‚Ää0.06), the former had a lower rate of liver transplantation (2% vs 18%, P‚Ää=‚Ää0.01). The rate of cirrhosis was lower in patients diagnosed with IBD before PSC (15% vs 31%, P‚Ää=‚Ää0.05). In this largest reported pediatric PSC cohort, liver transplantation rate at 10 years was lower than that reported in adults. ASC and PSC had similar biochemical abnormalities and degree of fibrosis at diagnosis. PSC that developed after IBD diagnosis had a milder course, possibly reflecting earlier disease detection or milder phenotype.","[0.0, 0.35, 0.78, 12.03]",2,2,...,Retrospective cohort study,30 years (1984 - 2014),120,30 years (1984 - 2014),Patients diagnosed with pediatric primary sclerosing cholangitis (PSC) between 1984 and 2014.,Not specified.,"- Median gamma glutamyl transferase decreased from 221 U/L to 104 U/L by 1 year post-diagnosis (P<0.0001), indicating improved liver function.\n- Transplant-free survival at 10 years was 89%.\n- Lower liver transplantation rate in PSC-IBD patients compared to those without IBD (2% vs 18%, P=0.01).\n- Cirrhosis rate was lower in patients diagnosed with IBD before PSC (15% vs 31%, P=0.05).",Not specified.,Primary sclerosing cholangitis (PSC),Not specified.
38,23433613,Gold Standard,'2 - the study's primary contribution centers on observing the time course of a rare disease',0.885,NaN,Natural history of low grade dysplasia in patients with primary sclerosing cholangitis and ulcerative colitis,"Patients with ulcerative colitis (UC) and primary sclerosing cholangitis (PSC) are at increased risk of colon cancer. The aim of this study was to determine the natural history of LGD and its progression to high grade dysplasia (HGD)/colorectal cancer (CRC) in PSC-UC patients. Ten PSC-UC patients with LGD who underwent surveillance colonoscopy from 1996 to 2011 were evaluated. Raised dysplasia was defined as a discrete raised lesion located in an area involved by either quiescent or active colitis that was endoscopically resected, while flat dysplasia was defined as the absence of documentation of a raised lesion. Of the 10

In [ ]:
df_temp

,PMID,Labeling_State,Correct_Label,Agreement,Explanation,TITLE,ABSTRACT,vector,curated,y,...,type of study,duration of the study,participants bumbers in study,data collection occur,inclusion criteria for the study,exclusion criteria for the study,clinical outcomes,differed between the treatments,rare disease associated with the study,Location where the NHS was conducted
0,27504812,Gold Standard,'2 - the study's primary contribution centers on observing the time course of a rare disease',0.900,NaN,The Natural History of Primary Sclerosing Cholangitis in Children: A Large Single-Center Longitudinal Cohort Study,"Data regarding pediatric primary sclerosing cholangitis (PSC) natural history are limited. We describe a large pediatric PSC cohort with longitudinal follow-up. The present study records review of pediatric patients with PSC diagnosed between 1984 and 2014. N‚Ää=‚Ää120 (63% M) ages 1 to 21 years (median 14 years) at diagnosis. 27% (31/113) had autoimmune sclerosing cholangitis (ASC), 24% had exclusive small duct PSC, METAVIR stage was F3-F4 in 41%. Eighty-one percent of patients with PSC had inflammatory bowel disease (IBD); most had ulcerative/indeterminate colitis (72/97), typically pancolitis (40/72). PSC-IBD was more common than ASC-IBD (85% vs 68%, P‚Ää=‚Ää0.03). Median follow-up was 3.7 years (interquartile range [IQR] 1.5, 6.9). Median gamma glutamyl transferase decreased from baseline of 221 U/L (IQR 110, 425) to 104 U/L by 1 year postdiagnosis ([IQR 18,229], P‚Ää<‚Ää0.0001), and then changed little. Mean fibrosis stage at diagnosis was 2.3‚Ää¬±‚Ää1.4 (N‚Ää=‚Ää91), and at 1 to 5 years was 2.6‚Ää¬±‚Ää1.3 (N‚Ää=‚Ää20). Transplant-free survival at 10 year was 89%; there were 6 liver transplants, 2 in patients with small duct PSC and 4 with diffuse large duct PSC. Although the cirrhosis rate was not significantly different in PSC with IBD versus without (22% vs 41%, P‚Ää=‚Ää0.06), the former had a lower rate of liver transplantation (2% vs 18%, P‚Ää=‚Ää0.01). The rate of cirrhosis was lower in patients diagnosed with IBD before PSC (15% vs 31%, P‚Ää=‚Ää0.05). In this largest reported pediatric PSC cohort, liver transplantation rate at 10 years was lower than that reported in adults. ASC and PSC had similar biochemical abnormalities and degree of fibrosis at diagnosis. PSC that developed after IBD diagnosis had a milder course, possibly reflecting earlier disease detection or milder phenotype.","[0.0, 0.35, 0.78, 12.03]",2,2,...,Retrospective longitudinal cohort study,The duration of the study is not specified in the abstract.,The answer is: 120,Here is the extracted information:\n\n**Title**: The Natural History of Primary Sclerosing Cholangitis in Children: A Large Single-Center Longitudinal Cohort Study.\n\n**Study Design**: Retrospective review\n\n**Number of Patients**: 120\n\n**Age Range**: 1 to 21 years (median 14 years) at diagnosis\n\n**Gender**: 63% Male\n\n**Location where the NHS was conducted**: Not specified in the abstract.\n\n**Clinical Outcomes**: Not specified.\n\n**Differed Between the Treatments**: Not applicable; treatments were not compared.\n\nLet me know if you need further assistance!,pediatric patients with PSC diagnosed between 1984 and 2014,Not specified in the abstract.,Not specified.,Not applicable; treatments were not compared.,Primary Sclerosing Cholangitis (PSC),Not specified in the abstract.
38,23433613,Gold Standard,'2 - the study's primary contribution centers on observing the time course of a rare disease',0.885,NaN,Natural history of low grade dysplasia in patients with primary sclerosing cholangitis and ulcerative colitis,"Patients with ulcerative colitis (UC) and primary sclerosing cholangitis (PSC) are at increased risk of colon cancer. The aim of this study was to determine the natural history of LGD and its progression to high grade dysplasia (HGD)/colorectal cancer (CRC) in PSC-UC patients. Ten PSC-UC patients with LGD who underwent surveillance colonoscopy from 1996 to 2011